<a href="https://colab.research.google.com/github/ik339/NTU-R-langauge/blob/main/Bioinformatical_analysis_of_genome_data_NTU_R_langauge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#bioinformatics assessment code
#2.0
> library(tidyverse)
> library(lubridate)
> library(stringdist)
> library(Biostrings) #First load in the correct libraries into R.
> setwd("/home/biouser/Downloads/assessment") #change directory to where the sequences are in R.


In [ ]:
#2.1
> euroseqMetaData <- read_csv("euroseq.csv")
and parsed using tidyverse functions. (Wickham et al., 2019).
> ymd(euroseqMetaData$Collection_Date)


In [ ]:
#2.2
> IncorrectDates <- is.na(ymd(euroseqMetaData$Collection_Date))  #Boolean vector identifying incomplete dates.
> notHumanHost <- euroseqMetaData %>% select(Host) %>% as_vector() %>% str_detect("Felis") #Boolean vector to identify Felis catus 
> NotHumanIncompleteDates <-  IncorrectDates | notHumanHost #combined Boolean vectors.


In [ ]:
#2.3
> euroseqMDFiltered <- euroseqMetaData %>% filter(!NotHumanIncompleteDates) #Make new tibble. Incorrect dates and non-human hosts removed.
> euroseqMDFiltered %>% type_convert() #Parse again
> ymd(euroseqMDFiltered$Collection_Date)  #check things are parsed correctly. 
> rm(“euroseqMetaData”) #remove old tibble from 2.1 


In [ ]:
#2.4
> euroseqMDFiltered %>% arrange(Collection_Date)
> euroseqMDFiltered %>% filter(Collection_Date == max(Collection_Date))   #Identify recent collected date


In [ ]:
#2.5
> euroseqMDFNew <- euroseqMDFiltered %>% mutate(Days = ymd(20200514) - ymd(Collection_Date)) %>% filter(Days >= 0)
#Subtracting each row's collection date from this recent collection point. Permanently add column “Days”, meaning days before recent collection date. 

In [ ]:
#3.1
#import fasta
> euroseqFastaStringSet <- readDNAStringSet("euroseq.fasta", format = "fasta")

In [ ]:
#3.2
> euroseqFSSFilt <- euroseqFastaStringSet[!NotHumanIncompleteDates] #Combined Boolean used.
> rm("euroseqFastaStringSet") #remove old unfiltered object.


In [ ]:
#3.3
> euroseqfasta  <- names(euroseqFSSFilt) %>% str_split(" \\|", n=2, simplify = TRUE) #parse sequence names
> colnames(euroseqfasta) <- c("Country", "Accession")	#add column names.
> euroseqNewfasta<- euroseqfasta %>% as_tibble()  #new tibble.


In [ ]:
#3.4
> euroseqFBA <- euroseqNewfasta %>% mutate(BaseAcc = str_extract(Accession, "[A-Za-z0-9_]+"))	 
#add base accession column, new tibble.

In [ ]:
#3.5
> euroDataJoined <- euroseqFBA%>%full_join(euroseqMDFNew, by = c("BaseAcc"="Accession")) 
# full outer join.new tibble.

In [ ]:
#3.6
> names(euroseqFSSFilt) <- str_c(names(euroseqFSSFilt), "|", euroDataJoined$Host)
> names(euroseqFSSFilt) <- str_c(names(euroseqFSSFilt), "|", euroDataJoined$Days) #add new collumns. 
> names(euroseqFSSFilt)%>%as_tibble()


In [ ]:
#3.7
> writeXStringSet(euroseqFSSFilt, "euroseqFSSFilt.fasta", format = "fasta")


In [ ]:
#4.1
conda activate msa 
cd Downloads/assessment #in bash terminal
cat euroFSSFilt.fasta mink3.fasta > combined.fasta #Combine fasta and mink.


In [ ]:
#4.2
mafft --auto --addfragments combined.fasta reference.fasta > aligned.fasta #open mafft

In [ ]:
#4.3. 
#Using the microseq library (Snipen and Hovde Liland, 2021)
#aligned sequence data were trimmed to remove all gaps at the 5’ and 3’ ends of the alignment: 

library(microseq) #load library in R
setwd("/home/biouser/Downloads/assessment")
Alignment <- readFasta("aligned.fasta")  #read it in R.
dim(Alignment) #check dimensions
str_length(Alignment$Sequence) #check length
TrimmedAlignment <- msaTrim(Alignment, gap.end=0, gap.mid=1) #trim str_length(TrimmedAlignment$Sequence)#check length.
#The reference sequence was removed and data written to file: 
WithoutReference <- TrimmedAlignment[2:dim(TrimmedAlignment)
writeFasta(WithoutReference, "trimmed.fasta", width = 80)
#Aligned and trimmed data were then used as input for phylogenetic analysis.
